## [추천시스템]
- TF-IDF와 코사인 유사도로 추천 시스템 구현
- 원리 : 유사한 내용 추천으로 유사도가 높은 것 찾기
    - 영화 줄거리(overview)가 비슷한 영화를 찾아주기

### 영화 추천 <hr>

In [35]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [36]:
### ===> 데이터 준비
datafile = '../data/movies_metadata.csv'

dataDF1 = pd.read_csv(filepath_or_buffer=datafile, low_memory=False)
dataDF1.info()
dataDF1.head(n=2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [37]:
import os
import sys

# 상위 폴더 경로 추가
sys.path.append(os.path.abspath(os.path.join(os.path.dirname('../func.py'), '..')))

# 모듈 임포트
from NLP import func

In [38]:
dataDF2 = dataDF1.head(10000)[['id', 'title', 'overview']]
func.funcDF_info(dataDF2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        10000 non-null  object
 1   title     10000 non-null  object
 2   overview  9971 non-null   object
dtypes: object(3)
memory usage: 234.5+ KB
DF.info :
None


DF.describe :
            id   title            overview
count    10000   10000                9971
unique    9997    9725                9955
top     105045  Hamlet  No overview found.
freq         2       4                  11


DF 결측치 파악 :
id           0
title        0
overview    29
dtype: int64




In [39]:
dataDF2.loc[:,'overview'] = dataDF2['overview'].fillna('')
func.funcDF_info(dataDF2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        10000 non-null  object
 1   title     10000 non-null  object
 2   overview  10000 non-null  object
dtypes: object(3)
memory usage: 234.5+ KB
DF.info :
None


DF.describe :
            id   title overview
count    10000   10000    10000
unique    9997    9725     9956
top     105045  Hamlet         
freq         2       4       29


DF 결측치 파악 :
id          0
title       0
overview    0
dtype: int64




# [Soynlp] 학습 기반 토크나이저
- 품사 태깅, 단어 토큰화 등을 지원하는 단어 토크나이저
- 비지도 학습으로 단어 토큰화 -> 데이터에 자주 등장하는 단어들을 단어로 분석
- 내부적으로 단어 점수 표로 동작

In [40]:
from konlpy.tag import Okt

In [41]:
tokenizer = Okt()

print(tokenizer.morphs("에이비식스 이대휘 1월 최애돌 기부 요정 입니다."))

# 형태소 분석 시 매개변수 stem, norm 둘 다 default는 False 
print(tokenizer.morphs("에이비식스 이대휘 1월 최애돌 기부 요정 입니다.", stem = True)) # 어간
print(tokenizer.morphs("에이비식스 이대휘 1월 최애돌 기부 요정 입니다.", norm = True)) # 정규화 

['에이', '비식스', '이대', '휘', '1월', '최애', '돌', '기부', '요정', '입니다', '.']
['에이', '비식스', '이대', '휘', '1월', '최애', '돌', '기부', '요정', '이다', '.']
['에이', '비식스', '이대', '휘', '1월', '최애', '돌', '기부', '요정', '입니다', '.']


In [42]:
# [soynlp] 사용 -> 말뭉치 데이터셋 
from urllib.request import urlretrieve
import os

In [43]:
filename = "../data/text_data.txt"

In [44]:
### ===> 학습 데이터 처리
from soynlp import DoublespaceLineCorpus # 한개로 통합된 문서 데이터를 분리하기 위함 
from soynlp.word import WordExtractor # 단어 추출 

In [45]:
### ===> 훈련 데이터 문서 분리
corpus = DoublespaceLineCorpus(filename)
print(f"훈련 데이터 문서 : {len(corpus)}개")

훈련 데이터 문서 : 30091개


In [46]:
# [주의] 실행 시 오래걸려서 주석으로 바꿔둠

### ===> SoyNLP 학습 진행
word_extractor = WordExtractor()

# 학습 진행하며 단어별 점수
word_extractor.train(sents=corpus)

# 단어별 점수표 추출
word_score_table = word_extractor.extract()



training was done. used memory 1.517 Gb
all cohesion probabilities was computed. # words = 223348
all branching entropies was computed # words = 361598
all accessor variety was computed # words = 361598


In [47]:
# 단어별 점수표 확인
for idx, key in enumerate(iterable=word_score_table):
    print(f'[{idx}]-{key}')
    if idx == 30:
        break

[0]-요
[1]-껍
[2]-렌
[3]-뜸
[4]-듣
[5]-꼴
[6]-젊
[7]-늦
[8]-액
[9]-끈
[10]-륜
[11]-덱
[12]-묶
[13]-백
[14]-파
[15]-태
[16]-넥
[17]-인
[18]-낼
[19]-홋
[20]-쿄
[21]-퀼
[22]-팰
[23]-작
[24]-칠
[25]-탑
[26]-땀
[27]-럽
[28]-키
[29]-자
[30]-엔


In [48]:
### ===> 응집 확률(cohesion probablity) : 내부 문자열(substring)이 얼마나 응집하여 자주 등장하는지를 판단하는 척도
# - 원리 : 문자열을 문자 단위로 분리, 왼쪽부터 순서대로 문자를 추가
#         각 문자열이 주어졌을 때 그 다음 문자가 나올 확률을 계산 / 누적곱 한 값
# - 값이 높을 수록 : 전체 코퍼스에서 이 문자열 시퀀스는 하나의 단어로 등장할 가능성 높음

In [49]:
word_score_table['바'].cohesion_forward

0

In [50]:
word_score_table['바다'].cohesion_forward

0.06393648140409527

In [51]:
word_score_table['바다에'].cohesion_forward

0.11518621707955429

In [52]:
### ===> SOYNLP의 L tokenzer
# - 띄어쓰기 단위로 나누 어절 토큰 : L 토큰 + R 토큰
#   (예 : '공원에' => '공원' + '에', '공부하는' => '공부 + 하는')
# - 분리 기준 : 점수가 가장 높은 L 토큰을 찾아내는 원리

In [53]:
from soynlp.tokenizer import LTokenizer

# 토큰으로 쪼개기 위한 L토큰 
scores = {word:score.cohesion_forward for word, score in word_score_table.items()}

l_tokenizer = LTokenizer(scores = scores)
l_tokenizer.tokenize('국제사회와 우리의 노력들로 범죄를 척결하자', flatten=False)

[('국제사회', '와'), ('우리', '의'), ('노력', '들로'), ('범죄', '를'), ('척결', '하자')]

In [54]:
### ===> 최대 점수 토크나이저
# - 띄어쓰기가 되지 않는 문장에서 점수가 높은 글자 시퀀스를 순차적으로 찾아내는 토크나이저
# - 띄어쓰기가 되어 있지 않은 묹아을 넣어서 점수를 통해 토큰화 된 결과

from soynlp.tokenizer import MaxScoreTokenizer

maxscore_tokenizer = MaxScoreTokenizer(scores=scores)       # MaxScoreTokenizer는 학습된 데이터셋을 바탕으로 하는 것
maxscore_tokenizer.tokenize('국제사회와우리의노력들로범죄를척결하자')

['국제사회', '와', '우리', '의', '노력', '들로', '범죄', '를', '척결', '하자']

### [3] TF-IDF와 Cosine <hr>

In [55]:
### TF-IDF : 단어들의 값 계산
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(dataDF2.overview)

### 코사인 유사도 : 두 개 matrix에 대한 비교 진행
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
cosine_sim, cosine_sim.shape

(array([[1.        , 0.01682915, 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.01682915, 1.        , 0.04871976, ..., 0.        , 0.01200997,
         0.        ],
        [0.        , 0.04871976, 1.        , ..., 0.        , 0.00735515,
         0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
         0.08838493],
        [0.        , 0.01200997, 0.00735515, ..., 0.        , 1.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.08838493, 0.        ,
         1.        ]]),
 (10000, 10000))

In [56]:
tfidf_matrix.toarray()[0]

array([0., 0., 0., ..., 0., 0., 0.])

In [57]:
print(f'TF-IDF 행렬의 크기(shape : {tfidf_matrix.shape}')
print(f'코사인 유사도 연관 결과 : {cosine_sim.shape}')

TF-IDF 행렬의 크기(shape : (10000, 32350)
코사인 유사도 연관 결과 : (10000, 10000)


In [58]:
cosine_sim[:10], dataDF2.loc[:10, 'title']

(array([[1.        , 0.01682915, 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.01682915, 1.        , 0.04871976, ..., 0.        , 0.01200997,
         0.        ],
        [0.        , 0.04871976, 1.        , ..., 0.        , 0.00735515,
         0.        ],
        ...,
        [0.        , 0.        , 0.00686749, ..., 0.0193363 , 0.        ,
         0.        ],
        [0.        , 0.10718403, 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ]]),
 0                       Toy Story
 1                         Jumanji
 2                Grumpier Old Men
 3               Waiting to Exhale
 4     Father of the Bride Part II
 5                            Heat
 6                         Sabrina
 7                    Tom and Huck
 8                    Sudden Death
 9                       GoldenEye
 10         The American President
 Name: title, dtype: obj

In [59]:
## 영화 제목 입력 ==> 해당 영화 인덱스 추출
(dataDF2.title == 'Father of the Bride Part II').argmax()

4

In [60]:
 ### 영화제목 : 인덱스
title_to_index = dict(zip(dataDF2.title, dataDF2.index))

### 원하는 영화 인덱스 찾기
title_index = 'GoldenEye'
select_idx = title_to_index[title_index]
select_idx

9

In [61]:
# # 기준 : 행 => 1개의 열이라도 비어 있으면 다 지워라
# dataDF1 = dataDF1.dropna(subset=['overview'])

In [68]:
# 모든 영화 유사도
sim_scores = list(enumerate(cosine_sim[select_idx]))
# print(sim_scores)

# 가장 유사한 영화 10개
sim_scores = sim_scores[1:11]
movie_indices = [idx[0] for idx in sim_scores]

# 가장 유사한 10개의 영화의 제목
pd.DataFrame(dataDF2.title.iloc[movie_indices])

,title
1,Jumanji
2,Grumpier Old Men
3,Waiting to Exhale
4,Father of the Bride Part II
5,Heat
6,Sabrina
7,Tom and Huck
8,Sudden Death
9,GoldenEye
10,The American President


In [69]:
dataDF1[['title','genres']].iloc[movie_indices]

,title,genres
1,Jumanji,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '..."
2,Grumpier Old Men,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ..."
3,Waiting to Exhale,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam..."
4,Father of the Bride Part II,"[{'id': 35, 'name': 'Comedy'}]"
5,Heat,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam..."
6,Sabrina,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '..."
7,Tom and Huck,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam..."
8,Sudden Death,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam..."
9,GoldenEye,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '..."
10,The American President,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam..."


In [72]:
import torch
import torchtext
import portalocker

In [74]:
print(torch.__version__)
print(torchtext.__version__)
print(portalocker.__version__)

2.2.2
0.17.2
2.8.2
